In [10]:
import pyarrow.parquet as pq
import pandas as pd
# import recipes.parquet into a pandas DataFrame
df: pd.DataFrame = pq.read_table('parquets/recipes.parquet').to_pandas()
df.columns

Index(['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions'],
      dtype='object')

There are many columns available to us here. But many of them are not particularly useful for analysis. Some examples of these are, cook time, prep time, date published, etc. We will drop these columns and keep only the ones that are useful for our analysis.

In [11]:
COLUMNS_TO_KEEP = [
    'RecipeId',
    'Name',
    'AuthorName',
    'Description',
    'RecipeCategory',
    'Keywords',
    'RecipeIngredientParts',
    'Calories',
    'FatContent',
    'SaturatedFatContent',
    'CholesterolContent',
    'SodiumContent',
    'CarbohydrateContent',
    'FiberContent',
    'SugarContent',
    'ProteinContent',
]

df_columns= df[COLUMNS_TO_KEEP]
df_columns.head()

,RecipeId,Name,AuthorName,Description,RecipeCategory,Keywords,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
0,38.0,Low-Fat Berry Blue Frozen Dessert,Dancer,Make and share this Low-Fat Berry Blue Frozen ...,Frozen Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...","[blueberries, granulated sugar, vanilla yogurt...",170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2
1,39.0,Biryani,elly9812,Make and share this Biryani recipe from Food.com.,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[saffron, milk, hot green chili peppers, onion...",1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4
2,40.0,Best Lemonade,Stephen Little,This is from one of my first Good House Keepi...,Beverages,"[Low Protein, Low Cholesterol, Healthy, Summer...","[sugar, lemons, rind of, lemon, zest of, fresh...",311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3
3,41.0,Carina's Tofu-Vegetable Kebabs,Cyclopz,This dish is best prepared a day in advance to...,Soy/Tofu,"[Beans, Vegetable, Low Cholesterol, Weeknight,...","[extra firm tofu, eggplant, zucchini, mushroom...",536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3
4,42.0,Cabbage Soup,Duckie067,Make and share this Cabbage Soup recipe from F...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...","[plain tomato juice, cabbage, onion, carrots, ...",103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3


For this project, we want to predict if a recipe will be healthy or not based on either the given keywords, or ingredients. But to check the feasibility of these approaches, we will first check how many unique keywords and ingredients are there in the dataset. If there are too many, then it will be difficult to use them for prediction.

In [4]:
keywords = set()
for kw in df_columns['Keywords']:
    keywords.update(kw)
len(keywords)

315

In [5]:
ingredients = set()
for ing in df_columns['RecipeIngredientParts']:
    ingredients.update(ing)
len(ingredients)

7368

Here we can see that keywords has a much more manageable list of 315 unique values, while ingredients has 7368 unique values. So we will use keywords for our analysis.

Next, we need to label all of the recipes as healthy or unhealthy. We do this using a simple heuristic involving the nutrient information given. This heuristic was based on the FDA's Daily Recommended Values for a 2000 calorie diet, which you can find [here](https://www.fda.gov/food/nutrition-facts-label/daily-value-nutrition-and-supplement-facts-labels). The FDA recommends that a healthy meal should contain between 5-20% of the daily value of each nutrient.

We will classify a recipe as health or unhealthy based on the following rules:
1. Calculate a "nutrient score" for each nutrient in the recipe. This value is 0 if it is within the 5-20% range. Otherwise, it will begin increasing as the value moves away from the 5-20% range.
    - This is concretely defined as `max(0, 0.1 * (x -5) * (x - 20))` where x is the percentage of the daily value of the nutrient in the recipe.
    - Notice that the non-zero function is a parabolic function that is 0 at 5 and 20, and increases as the value moves away from these points.
2. Sum the nutrient scores for each nutrient in the recipe.
3. Remove outliers.
4. Calculate the percentage of recipes that are within the DV for calories, and define this percentile as the threshold for a healthy recipe.

In [7]:
from typing import List

def nutrient_score(nutients: List[float], a: float = 0.1):
    return sum(map(lambda x: max(0, a * (x - 5) * (x - 20)), nutients))

NUTRIENT_COLUMNS = [
    'Calories',
    'FatContent',
    'SaturatedFatContent',
    'CholesterolContent',
    'SodiumContent',
    'CarbohydrateContent',
    'FiberContent',
#    'SugarContent', # There is no published DV for sugar
    'ProteinContent',
]

NUTRIENT_TARGET_DV = {
    'Calories': 2000,
    'FatContent': 78,
    'SaturatedFatContent': 20,
    'CholesterolContent': 300,
    'SodiumContent': 2300,
    'CarbohydrateContent': 275,
    'FiberContent': 28,
#    'SugarContent': 50, # 50g is the recommended daily limit for *added* sugar
    'ProteinContent': 50,
}

df_nutrients = df_columns.copy()
for column in NUTRIENT_COLUMNS:
    df_nutrients[column + 'DV'] = df_nutrients[column] / NUTRIENT_TARGET_DV[column]

df_nutrients['NutrientScore'] = df_nutrients[NUTRIENT_COLUMNS].apply(nutrient_score, axis=1)

Here, `MAX_SCORE` was determined by using the VSCode data wrangler to identify outliers.

In [8]:
# Drop values with nutrient score greater than MAX_SCORE
df_outliers = df_nutrients.copy()
print(len(df_outliers))
MAX_SCORE = 400_000
df_outliers = df_outliers[df_outliers['NutrientScore'] < MAX_SCORE]
print(len(df_outliers))

522517
479647


In [9]:
df_outliers.drop(columns=['Keywords', 'RecipeIngredientParts']).to_csv('recipes-dv.csv', index=False)

In [10]:
# Find the ratio of recipes with a calorie DV between 0.05 and 0.2 without creating a new df
calories = df_nutrients['CaloriesDV']
calories_filtered = calories[(calories > 0.05) & (calories < 0.2)]
calories_filtered_ratio = len(calories_filtered) / len(calories)
calories_filtered_ratio

0.5014152649578866

In [11]:
# The bottom calories_filtered_ratio% of the nutrient score column, are healthy
# Create a new boolean column called Healthy
df_healthy = df_nutrients.copy()
df_healthy['Healthy'] = df_healthy['NutrientScore'] < df_healthy['NutrientScore'].quantile(calories_filtered_ratio)

In [12]:
df_healthy.drop(columns=['Keywords', 'RecipeIngredientParts']).to_csv('recipes-healthy.csv', index=False)

Next we need to create the base vector for our model. This will be a vector of all of the unique keywords in the dataset, but in a concretely defined order.

In [13]:
def unique_elems(element_lists: List[List[str]]) -> List[str]:
    """Creates a list of all of the unique strings in the input lists
    This potentially be accomplished in a better way using sets, but sets 
    do not guarantee a specific order, which is necessary to keep the feature
    order consistent."""
    unique = []
    unique_set = set() # Uses a set to trade some extra memory usage to make `in` run in constant time
    for element_list in element_lists:
        for element in element_list:
            if element not in unique_set:
                unique_set.add(element)
                unique.append(element)
    return unique

In [14]:
# Get a list of all unique keywords and ingredients
# This is used to vectorize the keywords and ingredients columns
unique_keywords = unique_elems(df_healthy['Keywords'])
unique_ingredients = unique_elems(df_healthy['RecipeIngredientParts'])
# Quick check that the elements are actually unique
assert len(unique_keywords) == len(set(unique_keywords))
assert len(unique_ingredients) == len(set(unique_ingredients))

The `vectorize` function takes the base vector and the input elements to vectorize, and returns a vectorized version of the input elements. This is done by creating a vector of the same length as the base vector, and setting the value of each element to 1 if the element is in the input elements, and 0 otherwise.

In [15]:
def vectorize(unique_elems: List[str], input_elems: List[str]) -> List[int]:
    """Creates a vector of 1s and 0s, where a 1 indicates that the element at that index
    is in the input_elems list"""
    vector = [0] * len(unique_elems)
    for unique_elem in unique_elems:
        if unique_elem in input_elems:
            vector[unique_elems.index(unique_elem)] = 1
    return vector

In [ ]:
df_vectorized = df_healthy.copy()
df_vectorized['KeywordsVector'] = df_vectorized['Keywords'].apply(lambda x: vectorize(unique_keywords, x))

Saving the data to a parquet file and reloading it was a common action to act as checkpoints throughout the project.

In [17]:
df_vectorized.to_parquet('parquets/recipes-vectorized.parquet') # Save the vectorized DataFrame to a parquet file
# This is done in favor of a csv to maintain the list structure of the vector columns
# Gzip the parquet file to save space
import gzip
with open('parquets/recipes-vectorized.parquet', 'rb') as f:
    with gzip.open('parquets/recipes-vectorized.parquet.gz', 'wb') as f_gz:
        f_gz.writelines(f)

In [17]:
import pandas as pd
# If the file is not already un-gzipped, unzip it
import gzip
import os
if not os.path.exists('parquets/recipes-vectorized.parquet'):
    with open('parquets/recipes-vectorized.parquet', 'wb') as f:
        with gzip.open('parquets/recipes-vectorized.parquet.gz', 'rb') as f_gz:
            f.write(f_gz.read())
df_vectorized = pd.read_parquet('parquets/recipes-vectorized.parquet') # Read the vectorized DataFrame from the parquet file

In [18]:
df_vectorized.head()

,RecipeId,Name,AuthorName,Description,RecipeCategory,Keywords,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,...,FatContentDV,SaturatedFatContentDV,CholesterolContentDV,SodiumContentDV,CarbohydrateContentDV,FiberContentDV,ProteinContentDV,NutrientScore,Healthy,KeywordsVector
0,38.0,Low-Fat Berry Blue Frozen Dessert,Dancer,Make and share this Low-Fat Berry Blue Frozen ...,Frozen Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...","[blueberries, granulated sugar, vanilla yogurt...",170.9,2.5,1.3,...,0.032051,0.065,0.026667,0.012957,0.134909,0.128571,0.064,2599.240,True,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ..."
1,39.0,Biryani,elly9812,Make and share this Biryani recipe from Food.com.,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[saffron, milk, hot green chili peppers, onion...",1110.7,58.8,16.6,...,0.753846,0.830,1.242667,0.160174,0.306909,0.321429,1.268,147209.075,False,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, ..."
2,40.0,Best Lemonade,Stephen Little,This is from one of my first Good House Keepi...,Beverages,"[Low Protein, Low Cholesterol, Healthy, Summer...","[sugar, lemons, rind of, lemon, zest of, fresh...",311.1,0.2,0.0,...,0.002564,0.000,0.000000,0.000783,0.296364,0.014286,0.006,9434.649,True,"[0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,41.0,Carina's Tofu-Vegetable Kebabs,Cyclopz,This dish is best prepared a day in advance to...,Soy/Tofu,"[Beans, Vegetable, Low Cholesterol, Weeknight,...","[extra firm tofu, eggplant, zucchini, mushroom...",536.1,24.0,3.8,...,0.307692,0.190,0.000000,0.677652,0.233455,0.617857,0.586,266750.774,False,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,42.0,Cabbage Soup,Duckie067,Make and share this Cabbage Soup recipe from F...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...","[plain tomato juice, cabbage, onion, carrots, ...",103.6,0.4,0.1,...,0.005128,0.005,0.000000,0.417087,0.091273,0.171429,0.086,90502.116,False,"[0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

NOTE: The training done in this notebook is mostly for demonstration purposes. Please see the files in the `models` directory for the actual training and evaluation of the models.

In [4]:
df_sample = df_vectorized.sample(frac=0.1) # Sample 10% of the data

For our first attempt at a classification model, we will use a KNN model. We will use the `KNeighborsClassifier` from the `sklearn` library.

In [5]:
X = df_sample['KeywordsVector'].to_list()
y = df_sample['Healthy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [8]:
classifier.score(X_test, y_test)

0.6327796065222384

This score of 0.63 is not particularly good, but it is a good starting point. We will try to improve this score in the next steps. First we will try to do a grid search to find the best hyperparameters for the KNN model.

In [6]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(KNeighborsClassifier(), {'n_neighbors': [3, 5, 7, 9, 11]}, verbose=3, n_jobs=-1, cv=3)
grid_search.fit(X, y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 2/3] END ....................n_neighbors=11;, score=0.649 total time=45.4min
[CV 1/3] END ....................n_neighbors=11;, score=0.644 total time=46.1min
[CV 3/3] END ....................n_neighbors=11;, score=0.642 total time=45.5min


GridSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [11]}, verbose=3)

The performance has slightly increased by using more neighbors. We will now try to use a different model, the Support Vector Machine (SVM) model. We will use the `SVC` class from the `sklearn` library.

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
grid_search_svm = GridSearchCV(SVC(), {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}, verbose=3, n_jobs=-1, cv=3)
grid_search_svm.fit(X, y)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


Unfortunately, the SVM model was unable to train at all in a reasonable amount of time. It will be moved to a different file for training (see `models/svm.py`).

Next we will train an `XGBClasifier` model. This model is a gradient boosting model that is known for its high performance. We will use the `XGBClassifier` class from the `xgboost` library.

In [10]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
grid_search_xgb = GridSearchCV(XGBClassifier(), {'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7]}, verbose=3, n_jobs=-1, cv=3)
grid_search_xgb.fit(X, y)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3] END .....max_depth=3, n_estimators=100;, score=0.677 total time=  21.8s


/home/greg/recipe-classifier/venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 2/3] END .....max_depth=3, n_estimators=100;, score=0.674 total time=  20.7s
[CV 3/3] END .....max_depth=3, n_estimators=100;, score=0.677 total time=  19.0s
[CV 1/3] END .....max_depth=3, n_estimators=200;, score=0.679 total time=  36.1s
[CV 2/3] END .....max_depth=3, n_estimators=200;, score=0.675 total time=  31.3s
[CV 3/3] END .....max_depth=3, n_estimators=200;, score=0.679 total time=  33.4s
[CV 1/3] END .....max_depth=3, n_estimators=300;, score=0.679 total time=  45.7s
[CV 2/3] END .....max_depth=3, n_estimators=300;, score=0.676 total time=  45.0s
[CV 1/3] END .....max_depth=5, n_estimators=100;, score=0.680 total time=  22.0s
[CV 3/3] END .....max_depth=3, n_estimators=300;, score=0.680 total time=  46.3s
[CV 2/3] END .....max_depth=5, n_estimators=100;, score=0.676 total time=  20.9s
[CV 3/3] END .....max_depth=5, n_estimators=100;, score=0.680 total time=  20.7s
[CV 1/3] END .....max_depth=5, n_estimators=200;, score=0.680 total time=  34.3s
[CV 2/3] END .....max_depth=

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 300]},
             verbose=3)

The XGBClassifier model has a much better performance than the KNN model. We will now try an MLP model. We will use the `MLPClassifier` class from the `sklearn` library.

In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
grid_search_mlp = GridSearchCV(MLPClassifier(), {'hidden_layer_sizes': [(100,), (100, 100), (100, 100, 100)], 'alpha': [0.0001, 0.001, 0.01]}, verbose=3, n_jobs=-1, cv=3)
# Sample 10%
df_sample = df_vectorized.sample(frac=0.05)
X = df_sample['KeywordsVector'].to_list()
y = df_sample['Healthy']
grid_search_mlp.fit(X, y)
import pickle
with open('classifier_mlp.pkl', 'wb') as f:
    pickle.dump(grid_search_mlp, f)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


/home/greg/recipe-classifier/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/home/greg/recipe-classifier/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/home/greg/recipe-classifier/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/home/greg/recipe-classifier/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/home/greg/recipe-classifier/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  wa

KeyboardInterrupt: 

In [ ]:
import pickle
with open('classifier.pkl', 'wb') as f:
    pickle.dump(grid_search, f)

with open('classifier_svm.pkl', 'wb') as f:
    pickle.dump(grid_search_svm, f)

with open('classifier_xgb.pkl', 'wb') as f:
    pickle.dump(grid_search_xgb, f)

with open('classifier_mlp.pkl', 'wb') as f:
    pickle.dump(grid_search_mlp, f)

To help reduce dimensionality, we will use 2 approaches. First we will do manual corpus reduction.

This is done by first removing words from the corpus that do not have any bearing on health.

Then, similar keywords will be grouped together. Ex:

Countries, for the most part, get grouped together into their respective continents. (Some exceptions are made for countries whose cuisine is very different from the rest of the continent)

Fruits, vegetables, fish, chicken, beef etc. are grouped together into their respective food groups.

In [19]:
# Corpus filtering
# Write keywords to a file
with open('keywords.txt', 'w') as f:
    for keyword in unique_keywords:
        f.write(keyword + '\n') if keyword else None

This keywords file was then manually reduced following the steps above into a new file called combinations.txt

In [20]:
# Read combinations.txt
# Split by double newline
# Each row between double newline is a combination
# The first row is the category name but can also be a keyword
combinations = []
with open('combinations.txt', 'r') as f:
    combinations = f.read().split('\n\n')

# Create a dictionary of categories to keywords
category_keywords = {}
for combination in combinations:
    lines = combination.split('\n')
    category = lines[0]
    keywords = lines[:] # Copy the list (Categories can also be keywords)
    category_keywords[category] = keywords

In [21]:
df_categorized_keywords = df_healthy.copy()

def get_keyword_categories(keywords: List[str], category_keywords: dict) -> List[str]:
    """
    Given a list of keywords and a dictionary of categories to keywords, return the categories that the keywords belong to
    """
    categories = []
    for keyword in keywords:
        for category, category_keywords_list in category_keywords.items():
            if keyword in category_keywords_list:
                if category not in categories:
                    categories.append(category)
    return categories

df_categorized_keywords['KeywordCategoriesVector'] = df_categorized_keywords['Keywords'].apply(lambda x: vectorize(list(category_keywords.keys()), get_keyword_categories(x, category_keywords)))

In [25]:
# Export categorized keywords to a parquet file
df_categorized_keywords.to_parquet('recipes-categorized.parquet') # Save the categorized DataFrame to a parquet file

The second approach is to use Principal Component Analysis (PCA) to reduce the dimensionality of the data.

In [28]:
# PCA on original data
from sklearn.decomposition import PCA
pca_99 = PCA(0.99)
X_99 = df_vectorized['KeywordsVector'].to_list()
X_pca_99 = pca_99.fit_transform(X_99)

pca_95 = PCA(0.95)
X_95 = df_vectorized['KeywordsVector'].to_list()
X_pca_95 = pca_95.fit_transform(X_95)

pca_90 = PCA(0.90)
X_90 = df_vectorized['KeywordsVector'].to_list()
X_pca_90 = pca_90.fit_transform(X_90)

print(len(df_vectorized['KeywordsVector'][0]))
print(pca_99.n_components_)
print(pca_95.n_components_)
print(pca_90.n_components_)

315
150
84
59


In [30]:
# Export the X_99, X_95 and X_90 to a parquet files
df_pca_99 = pd.DataFrame(X_pca_99)
df_pca_95 = pd.DataFrame(X_pca_95)
df_pca_90 = pd.DataFrame(X_pca_90)
df_pca_99.to_parquet('parquets/recipes-pca-99.parquet')
df_pca_95.to_parquet('parquets/recipes-pca-95.parquet')
df_pca_90.to_parquet('parquets/recipes-pca-90.parquet')

Eventually, we decided to not use the combined keywords file, as it was not very effective in improving the model's performance, and it was incredibly time consuming to create and maintain.

It was at this point that we moved all of our model training to separate files in the `models` directory. Below is some analysis of the models that were trained in those files.

In [2]:
import pandas as pd
import pickle
grid_search_mlp = pickle.load(open('pkls/classifier_mlp.pkl', 'rb'))

print(grid_search_mlp.best_params_)
print(grid_search_mlp.best_score_)
pd.DataFrame(grid_search_mlp.cv_results_)

{'alpha': 0.01, 'hidden_layer_sizes': (100,)}
0.6653384776734109


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,577.981225,26.571960,0.751856,0.134677,0.0001,"(100,)","{'alpha': 0.0001, 'hidden_layer_sizes': (100,)}",0.653825,0.658108,0.658613,0.656849,0.002148,3
1,966.961032,142.379300,1.140953,0.226300,0.0001,"(100, 100)","{'alpha': 0.0001, 'hidden_layer_sizes': (100, ...",0.642365,0.636463,0.638645,0.639157,0.002437,8
2,1746.250128,256.875059,2.255075,0.499300,0.0001,"(100, 100, 100)","{'alpha': 0.0001, 'hidden_layer_sizes': (100, ...",0.640195,0.640574,0.641251,0.640673,0.000437,7
3,682.099699,29.544583,0.951196,0.182693,0.001,"(100,)","{'alpha': 0.001, 'hidden_layer_sizes': (100,)}",0.657201,0.655272,0.659922,0.657465,0.001908,2
4,1348.390028,23.076754,2.147042,0.154684,0.001,"(100, 100)","{'alpha': 0.001, 'hidden_layer_sizes': (100, 1...",0.644880,0.642549,0.640643,0.642690,0.001733,6
5,2384.282684,267.379201,3.626477,0.447349,0.001,"(100, 100, 100)","{'alpha': 0.001, 'hidden_layer_sizes': (100, 1...",0.637783,0.639127,0.639242,0.638717,0.000662,9
6,742.967716,10.978124,0.932565,0.072057,0.01,"(100,)","{'alpha': 0.01, 'hidden_layer_sizes': (100,)}",0.663861,0.665411,0.666743,0.665338,0.001178,1
7,2181.766516,117.124281,3.969333,0.274357,0.01,"(100, 100)","{'alpha': 0.01, 'hidden_layer_sizes': (100, 100)}",0.651873,0.651574,0.649037,0.650828,0.001273,5
8,3293.751387,284.404037,5.973990,0.088487,0.01,"(100, 100, 100)","{'alpha': 0.01, 'hidden_layer_sizes': (100, 10...",0.653986,0.653894,0.654675,0.654185,0.000348,4


In [3]:
grid_search_svm = pickle.load(open('pkls/classifier_svm.pkl', 'rb'))

print(grid_search_svm.best_params_)
print(grid_search_svm.best_score_)
pd.DataFrame(grid_search_svm.cv_results_)

{'C': 1, 'kernel': 'rbf'}
0.6752467293664397


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,426.934598,32.069114,122.611640,7.557407,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",0.669767,0.658150,0.662341,0.663419,0.004803,6
1,489.657964,34.231968,153.599291,10.300628,0.1,poly,"{'C': 0.1, 'kernel': 'poly'}",0.664772,0.655394,0.658896,0.659688,0.003869,8
2,473.845591,35.527259,470.018854,49.011955,0.1,rbf,"{'C': 0.1, 'kernel': 'rbf'}",0.680273,0.666992,0.667451,0.671572,0.006155,2
3,456.437346,22.304599,121.961729,5.007897,0.1,sigmoid,"{'C': 0.1, 'kernel': 'sigmoid'}",0.597772,0.602744,0.596314,0.598944,0.002753,13
4,556.228030,54.905253,130.071136,27.475192,1,linear,"{'C': 1, 'kernel': 'linear'}",0.668848,0.658322,0.663892,0.663687,0.004300,5
5,1074.288121,87.241463,138.573233,24.355305,1,poly,"{'C': 1, 'kernel': 'poly'}",0.668217,0.659528,0.661595,0.663113,0.003706,7
6,791.329003,43.509996,390.336230,130.129802,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.683144,0.670265,0.672332,0.675247,0.005648,1
7,368.949031,76.032076,88.755938,17.282088,1,sigmoid,"{'C': 1, 'kernel': 'sigmoid'}",0.582443,0.583797,0.578745,0.581662,0.002135,14
8,1846.930208,56.646181,115.349479,21.161190,10,linear,"{'C': 10, 'kernel': 'linear'}",0.669250,0.658265,0.663892,0.663802,0.004485,4
9,3286.963637,91.600119,128.140681,21.952162,10,poly,"{'C': 10, 'kernel': 'poly'}",0.640831,0.636677,0.639318,0.638942,0.001717,10


In [4]:
grid_search_xgb = pickle.load(open('pkls/classifier_xgb.pkl', 'rb'))

print(grid_search_xgb.best_params_)
print(grid_search_xgb.best_score_)
pd.DataFrame(grid_search_xgb.cv_results_)

{'max_depth': 7, 'n_estimators': 100}
0.6773993523643295


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,15.142815,2.203515,0.482310,0.061376,3,100,"{'max_depth': 3, 'n_estimators': 100}",0.674758,0.673920,0.675930,0.674869,0.000824,9
1,23.844930,4.592548,0.942388,0.168291,3,200,"{'max_depth': 3, 'n_estimators': 200}",0.675103,0.674586,0.676860,0.675516,0.000973,7
2,31.516101,2.040749,1.021727,0.286612,3,300,"{'max_depth': 3, 'n_estimators': 300}",0.675872,0.675665,0.678364,0.676634,0.001226,5
3,14.044438,2.113280,0.743874,0.125889,5,100,"{'max_depth': 5, 'n_estimators': 100}",0.676125,0.676228,0.677882,0.676745,0.000805,3
4,26.008682,2.916427,1.077635,0.233605,5,200,"{'max_depth': 5, 'n_estimators': 200}",0.676492,0.676848,0.678375,0.677239,0.000817,2
5,34.338681,7.480582,1.929509,0.173466,5,300,"{'max_depth': 5, 'n_estimators': 300}",0.676182,0.676504,0.677480,0.676722,0.000552,4
6,17.600952,2.367708,0.958938,0.264719,7,100,"{'max_depth': 7, 'n_estimators': 100}",0.676676,0.676814,0.678708,0.677399,0.000927,1
7,24.617706,4.644564,1.400888,0.245499,7,200,"{'max_depth': 7, 'n_estimators': 200}",0.675907,0.676320,0.677319,0.676515,0.000593,6
8,34.999473,6.280626,2.151429,0.656923,7,300,"{'max_depth': 7, 'n_estimators': 300}",0.674908,0.675149,0.676010,0.675355,0.000473,8


In [2]:
import pandas as pd
# Load and join recipes.parquet and recipes-vectorized.parquet by RecipeId
df = pd.read_parquet('parquets/recipes.parquet')
df_vectorized = pd.read_parquet('parquets/recipes-vectorized.parquet')
# Drop any columns in df that are already in df_vectorized (except for RecipeId)
df.drop(columns=[col for col in df.columns if col in df_vectorized.columns and col != 'RecipeId'], inplace=True)
df = df.merge(df_vectorized, on='RecipeId')
# Save the joined DataFrame to a parquet file
df.to_parquet('parquets/recipes-joined.parquet')

In [7]:
# Load all pickled models from the pkls directory and print their best parameters and scores
# Also save the best GridSearchCV object to a variable
import pickle
import os
import pandas as pd
import numpy as np
from typing import List
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

best_score = -np.inf
best_model = None

for filename in os.listdir('pkls'):
    if filename.endswith('.pkl'):
        with open(f'pkls/{filename}', 'rb') as f:
            grid_search = pickle.load(f)
            print(f'{filename}:')
            print(grid_search.best_params_)
            print(grid_search.best_score_)
            print()
            if isinstance(grid_search.best_estimator_, SVC):
                grid_search_svm = grid_search
            elif isinstance(grid_search.best_estimator_, XGBClassifier):
                grid_search_xgb = grid_search
            elif isinstance(grid_search.best_estimator_, MLPClassifier):
                grid_search_mlp = grid_search
            if grid_search.best_score_ > best_score:
                best_score = grid_search.best_score_
                best_model = grid_search.best_estimator_
pickle.dump(best_model, open('pkls/best_model.pkl', 'wb'))

classifier_mlp.pkl:
{'alpha': 0.01, 'hidden_layer_sizes': (100,)}
0.6653384776734109

classifier_svm.pkl:
{'C': 1, 'kernel': 'rbf'}
0.6752467293664397

classifier_xgb.pkl:
{'max_depth': 7, 'n_estimators': 100}
0.6773993523643295

classifier_svm_pca_90.pkl:
{'C': 1, 'kernel': 'rbf'}
0.6713013209744002

classifier_xgb_pca_99.pkl:
{'max_depth': 3, 'n_estimators': 300}
0.6736316762855644

classifier_xgb_pca_95.pkl:
{'max_depth': 3, 'n_estimators': 300}
0.6722690413664019

classifier_xgb_pca_90.pkl:
{'max_depth': 3, 'n_estimators': 300}
0.6708068885129957



In [3]:
df_joined = pd.read_parquet('parquets/recipes-joined.parquet')
print(df_joined.columns)
df_joined.head()

Index(['RecipeId', 'AuthorId', 'CookTime', 'PrepTime', 'TotalTime',
       'DatePublished', 'Images', 'RecipeIngredientQuantities',
       'AggregatedRating', 'ReviewCount', 'RecipeServings', 'RecipeYield',
       'RecipeInstructions', 'Name', 'AuthorName', 'Description',
       'RecipeCategory', 'Keywords', 'RecipeIngredientParts', 'Calories',
       'FatContent', 'SaturatedFatContent', 'CholesterolContent',
       'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent',
       'ProteinContent', 'CaloriesDV', 'FatContentDV', 'SaturatedFatContentDV',
       'CholesterolContentDV', 'SodiumContentDV', 'CarbohydrateContentDV',
       'FiberContentDV', 'ProteinContentDV', 'NutrientScore', 'Healthy',
       'KeywordsVector'],
      dtype='object')


,RecipeId,AuthorId,CookTime,PrepTime,TotalTime,DatePublished,Images,RecipeIngredientQuantities,AggregatedRating,ReviewCount,...,FatContentDV,SaturatedFatContentDV,CholesterolContentDV,SodiumContentDV,CarbohydrateContentDV,FiberContentDV,ProteinContentDV,NutrientScore,Healthy,KeywordsVector
0,38.0,1533,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,[https://img.sndimg.com/food/image/upload/w_55...,"[4, 1⁄4, 1, 1]",4.5,4.0,...,0.032051,0.065,0.026667,0.012957,0.134909,0.128571,0.064,2599.240,True,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ..."
1,39.0,1567,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,[https://img.sndimg.com/food/image/upload/w_55...,"[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...",3.0,1.0,...,0.753846,0.830,1.242667,0.160174,0.306909,0.321429,1.268,147209.075,False,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, ..."
2,40.0,1566,PT5M,PT30M,PT35M,1999-09-05 19:52:00+00:00,[https://img.sndimg.com/food/image/upload/w_55...,"[1 1⁄2, 1, None, 1 1⁄2, None, 3⁄4]",4.5,10.0,...,0.002564,0.000,0.000000,0.000783,0.296364,0.014286,0.006,9434.649,True,"[0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,41.0,1586,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00+00:00,[https://img.sndimg.com/food/image/upload/w_55...,"[12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1⁄2, 1⁄...",4.5,2.0,...,0.307692,0.190,0.000000,0.677652,0.233455,0.617857,0.586,266750.774,False,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,42.0,1538,PT30M,PT20M,PT50M,1999-09-19 06:19:00+00:00,[https://img.sndimg.com/food/image/upload/w_55...,"[46, 4, 1, 2, 1]",4.5,11.0,...,0.005128,0.005,0.000000,0.417087,0.091273,0.171429,0.086,90502.116,False,"[0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
